# Advent of Code 2017

Inspired by [Peter Norvig](https://github.com/norvig/pytudes/blob/master/ipynb/Advent%20of%20Code.ipynb), I am going to try the [Advent of Code 2017](http://adventofcode.com/2017) using a Jupyter notebook.

First, I'll import some standard modules.

In [8]:
import collections
import functools
import itertools as its
import math
import operator
import re
import string

I'll also gather up general purpose functions here.

In [9]:
def input(day):
    'Load the input for `day`'
    return open(f'input/{day}').read()

def ilen(xs):
    'Return the length of an iterable'
    return sum(1 for _ in iter(xs))

def last(xs):
    'Return the last element of an iterable sequence.'
    xs = iter(xs)
    return collections.deque(xs, maxlen=1)[0]

def ints(s):
    'Return a list of integer tokens in `s`.'
    return [int(x) for x in re.findall(r'-?\d+', s)]

def array(data):
    'Convert the data into a list of lists of ints.'
    return [ints(line) for line in data.splitlines()]

def swap(xs, i, j):
    'Swap items in a sequence.'
    xs[i], xs[j] = xs[j], xs[i]

### [Day 1](http://adventofcode.com/2017/day/1)
Calculate a "captcha" from an input sequence.

In [5]:
def captcha(xs):
    '''Sum the digits in xs which match the next in the sequence.

    Note: the sequence is circular, so the final element is succeeded
    by the first.
    1122 => 1 + 2 = 3
    1111 => 1 + 1 + 1 + 1 = 4
    91212129 => 9
    '''
    return sum(int(t) for t, n in zip(xs, xs[1:] + xs[:1]) if t==n)

captcha(input(1))

1089

For the second part of the puzzle, rather than look at the next digit, we should look at the one halfway around. Let's adapt `captcha()` to use any `stride`.

In [6]:
def captcha(xs, stride=1):
    '''Sum the digits in xs which match the digit `stride` ahead in the sequence.'''
    return sum(int(t) for t, n in zip(xs, xs[stride:] + xs[:stride]) if t==n)

def captcha_part2(xs):
    assert len(xs) % 2 == 0
    return captcha(xs, len(xs)//2)

def test_captcha():
    assert captcha('1122') == 3
    assert captcha('1111') == 4
    assert captcha('91212129') == 9
    assert captcha_part2('1212') == 6
    assert captcha_part2('1221') == 0
    assert captcha_part2('123123') == 12
    assert captcha_part2('12131415') == 4

test_captcha()

captcha_part2(input(1))

1156

### [Day 2](http://adventofcode.com/2017/day/2)
The task is to find the checksum of a grid of letters. In part one, the checksum is the sum of the spread of values taken by each row.

In [26]:
def checksum(xs):
    return sum(max(row) - min(row) for row in xs)

spreadsheet = array

data = '''\
5 1 9 5
7 5 3
2 4 6 8'''

assert checksum(spreadsheet(data)) == 18

checksum(spreadsheet(input(2)))

48357

In part two, we are told a there's a pair of numbers on each line where the second evenly divides the first. The checksum is the result of summing the result of dividing the first by the second on each row.
We can do this by considering _all_ pairs `(x, y)` on each row, and adding a term of `x//y` if `y` evenly divides `x` or `0` otherwise.

In [8]:
def term(x, y):
    return 0 if x % y else x//y

def checksum_part2(xs):
    return sum(sum(its.starmap(term, its.permutations(row, 2))) for row in xs)

data = '''\
5 9 2 8
9 4 7 3
3 8 6 5'''

assert checksum_part2(spreadsheet(data)) == 9

checksum_part2(spreadsheet(input(2)))

351

### [Day 3](http://adventofcode.com/2017/day/3)
This puzzle is about navigating a spiral pattern on a two dimensional grid. Let's use complex numbers to represent points and directions on this grid. Turning left is done by multiplying by `1j`.

In [9]:
input3 = 368078

def spiral(pos):
    'Generate positions "spiralling" from the supplied `pos`.'
    # Spiral anti-clockwise, heading W, N, E, S
    # That is, step 1, 1, 2, 2, 3, 3... in direction W, N, E, S, W, N...
    direction = 1
    def rotate_anticlockwise(direction): 
        return direction * 1j
    for step2 in its.count(2):
        step = step2//2
        yield from (pos + direction * s for s in range(step))
        pos += direction * step
        direction = rotate_anticlockwise(direction)

def test_spiral():
    s = spiral(0)
    assert next(s) == 0
    assert next(s) == 1
    assert next(s) == 1+1j
    assert next(s) == 1j
    assert next(s) == -1+1j
    assert next(s) == -1
    assert next(s) == -1-1j

test_spiral()

In part one, we must calculate the Manhattan Distance of the place we get to after a number of steps from the starting point.

In [10]:
def distance_to_origin(steps):
    s = spiral(0)
    pos = next(its.islice(s, steps - 1, None))
    return math.fabs(pos.real) + math.fabs(pos.imag)

def test_distance_to_origin():
    assert distance_to_origin(1) == 0
    assert distance_to_origin(12) == 3
    assert distance_to_origin(23) == 2
    assert distance_to_origin(1024) == 31

test_distance_to_origin()

distance_to_origin(input3)

371.0

In part 2, we walk the spiral again, this time filling each square we visit with the sum of its neighbouring values, initialising the first square to `1`.

In [11]:
def neighbours(p):
    "Return `p`'s neighbours in the grid."
    ns = {1, 1+1j, 1j, -1+1j, -1, -1-1j, -1j, 1-1j}
    return {p + n for n in ns}

def sum_adjacent_squares():
    '''Part two of the puzzle - each square is the sum of its neighbours.'''
    values = {}
    for p in spiral(0):
        values[p] = 1 if p == 0 else sum(values.get(n, 0) for n in neighbours(p))
        yield values[p]

next(v for v in sum_adjacent_squares() if v > input3)

369601

### [Day 4](http://adventofcode.com/2017/day/4) High-Entropy Passphrases
Count up the valid passphrases.

In [12]:
def valid_passphrase(p, key=lambda w: w):
    '''Is the passphrase `p` valid?

    Returns True iff no words in the passphrase have the same key.
    '''
    words = p.split()
    return len(words) == len({key(w) for w in words})

def test_valid_passphrase():
    assert valid_passphrase('aa bb cc dd ee')
    assert not valid_passphrase('aa bb cc dd aa')
    assert valid_passphrase('aa bb cc dd aaa')

test_valid_passphrase()

sum(valid_passphrase(p) for p in input(4).splitlines())

451

In the second part, none of the words in the passphrases can be anagrams of each other.

In [13]:
truths = '''\
abcde fghij is valid
abcde xyz ecdab is not valid
a ab abc abd abf abj is valid
iiii oiii ooii oooi oooo is valid
oiii ioii iioi iiio is not valid'''

def anagram_key(w):
    return tuple(sorted(w))

def test_anagram_passphrases():
    for t in truths.splitlines():
        p, _, v = t.partition(' is ')
        assert v in {'valid', 'not valid'}
        assert valid_passphrase(p, anagram_key) == (v == 'valid')

test_anagram_passphrases()

sum(valid_passphrase(p, anagram_key) for p in input(4).splitlines())

223

### [Day 5](http://adventofcode.com/2017/day/5) Twisty Trampolines
The program is a list of jump offsets. Each offset is incremented after it's been applied.

In [14]:
def steps(jumps, pos=0):
    'Generate PC positions for machine in day 5.'
    jumps = list(jumps)
    while 0 <= pos < len(jumps):
        yield pos
        j = jumps[pos]
        jumps[pos] += 1
        pos += j

assert ilen(steps([0, 3, 0, 1, -3])) == 5
jumps = [int(w) for w in input(5).split()]
ilen(steps(jumps))

354121

In part 2, the rules for updating the offsets are more complicated.

In [15]:
def steps_part2(jumps, pos=0):
    'Generate PC positions for machine in day 5, part b.'
    jumps = list(jumps)
    while 0 <= pos < len(jumps):
        yield pos
        j = jumps[pos]
        jumps[pos] += 1 if j < 3 else -1
        pos += j

assert ilen(steps_part2([0, 3, 0, 1, -3])) == 10
ilen(steps_part2(jumps))

27283023

### [Day 6](http://adventofcode.com/2017/day/6) Memory reallocation
The program redistributes memory in a list of memory banks, until a repeated pattern is detected. Part one of the puzzle asks for the time until this happens -- acheived by keeping a set of patterns seen so far. Part two also asks for the length of the cycle detected, which can be done using a `{pattern: time}` dictionary.

In [16]:
def memory_banks(banks):
    'Generate the sequence of values held in the supplied memory banks.'
    B = len(banks)
    while True:
        yield banks
        # Find the position of the first bank with the maximum value
        M, p = max((b, -i) for i, b in enumerate(banks))
        p = -p
        # Now spread the contents of this bank evenly
        banks[p] = 0
        for i in range(p+1, p+1+M):
            banks[i % B] += 1

def cycle_until_repeat(banks):
    '''Cycle the memory banks until we see a pattern we've already seen.

    Returns: the time taken to see a repeat and the time since the
    value was first seen.
    '''
    seen = {}
    for t, b in enumerate(memory_banks(banks)):
        b = tuple(b)
        if b in seen:
            return t, t - seen[b]
        else:
            seen[b] = t

assert cycle_until_repeat([0, 2, 7, 0]) == (5, 4)
cycle_until_repeat([int(w) for w in input(6).split()])

(14029, 2765)

### [Day 7](http://adventofcode.com/2017/day/7) Recursive Circus
Tree processing.

In [17]:
example_data = '''\
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)'''

def read_graph_and_weights(data):
    '''Returns the graph & weights given data as shown in the example.

    Returns:
      - a dict mapping parent to children
      - a dict mapping node to weight
    '''
    parse = re.compile('\w+').findall
    rows = [parse(row) for row in data.splitlines()]
    return ({node: set(subs)
             for node, _, *subs in rows},
            {node: int(weight)
             for node, weight, *_ in rows})

def flatten(xss):
    return its.chain.from_iterable(xss)

def root(t):
    'Find the root of a tree, `t`'
    top = set(t) - set(flatten(t.values()))
    assert len(top) == 1
    return top.pop()

t_example, w_example = read_graph_and_weights(example_data)
root(t_example)

'tknk'

In [18]:
t_day7, w_day7 = read_graph_and_weights(input(7))
print(root(t_day7))

ahnofa


In [19]:
def transpose_tree(t):
    return {c: p for p, children in t.items() for c in children}

def balance_tree(t, weights):
    tt = transpose_tree(t)
    def weight(p):
        return weights[p] + sum(map(weight, t[p]))
    def siblings(c):
        return t[tt[c]] - {c}
    def wrong_weight(p):
        return weight(p) not in map(weight, siblings(p))
    
    # Descend the tree following the nodes of wrong weight
    wrong = root(t)
    while True:
        new_wrong = {c for c in t[wrong] if wrong_weight(c)}
        assert len(new_wrong) <= 1
        if new_wrong:
            wrong = new_wrong.pop()
        else:
            adjust = weight(siblings(wrong).pop()) - weight(wrong)
            return weights[wrong] + adjust

assert balance_tree(t_example, w_example) == 60
balance_tree(t_day7, w_day7)

802

### [Day 8](http://adventofcode.com/2017/day/8) I Heard You Like Registers
Implement a sequence of instructions. Since the instructions are almost Python, we can use `exec` and `eval`.

In [20]:
example_instructions = '''\
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10'''

def eval_cond(cond, registers):
    'Evaluate the condition, modifying the registers.'
    cond = re.sub(r'([a-zA-Z]+)', r'registers["\1"]', cond)
    return eval(cond)

def exec_cmd(cmd, registers):
    'Evaluate the command, modifying the registers.'
    ops = {'inc': '+=', 'dec': '-='}
    reg, op, val = cmd.split()
    cmd = f'registers["{reg}"] {ops[op]} {val}'
    exec(cmd)

def execute_instruction(instruction, registers):
    'Executes `line`, modifying `registers`.'
    cmd, _, cond = instruction.partition(' if ')
    if eval_cond(cond, registers):
        exec_cmd(cmd, registers)

def run_program(data):
    'Runs the program, yielding register state after each step.'
    registers = collections.defaultdict(int)
    for instruction in data.splitlines():
        execute_instruction(instruction, registers)
        yield registers

last(run_program(example_instructions))

defaultdict(int, {'a': 1, 'b': 0, 'c': -10})

In [21]:
max(last(run_program(input(8))).values())

4416

In [22]:
max(max(regs.values()) for regs in run_program(input(8)))

5199

### [Day 9](http://adventofcode.com/2017/day/9) Stream Processing

In [23]:
def skip_garbage(s, pos):
    '''Returns the position in `s` after the garbage ends
    & the length of the garbage.

    pos - the index of a garbage start char <
    '''
    n = 0
    assert s[pos] == '<'
    while s[pos] != '>':
        if s[pos] == '!':
            pos += 2
        else:
            pos += 1
            n += 1
    return pos + 1, n - 1 # Advance past >, don't count initial <

def test_skip_garbage():
    garbage = (('<>', 0),
               ('<random characters>', 17),
               ('<<<<>', 3),
               ('<{!>}>', 2),
               ('<!!>', 0),
               ('<!!!>>', 0),
               ('<{o"i!a,<{i<a>', 10))
    assert all(skip_garbage(g, 0) == (len(g), n) for g, n in garbage)

test_skip_garbage()

def count_groups_and_garbage(s):
    b, e = 0, len(s)
    assert s[b] == '{'
    groups, garbage = 0, 0
    group_depth = 0
    while b != e:
        if s[b] == '<':
            b, n = skip_garbage(s, b)
            garbage += n
        else:
            group_depth += {'{': 1, '}': -1, ',': 0}[s[b]]
            if s[b] == '{':
                groups += group_depth
            b += 1
    assert group_depth == 0
    return groups, garbage

def test_count_groups_and_garbage():
    tests = (('{}', 1),
             ('{{{}}}', 6),
             ('{{},{}}', 5),
             ('{{{},{},{{}}}}', 16),
             ('{<a>,<a>,<a>,<a>}', 1),
             ('{{<ab>},{<ab>},{<ab>},{<ab>}}', 9),
             ('{{<!!>},{<!!>},{<!!>},{<!!>}}', 9),
             ('{{<a!>},{<a!>},{<a!>},{<ab>}}', 3),)
    assert all(count_groups_and_garbage(s)[0] == n for s, n in tests)

test_count_groups_and_garbage()
count_groups_and_garbage(input(9).strip())

(23588, 10045)

### [Day 10](http://adventofcode.com/2017/day/10) Knot Hash
Rather than fiddle with reversing elements which wrap around the end of the list, keep rotating the (circular) list so we can simply reverse the elements at its front. A variable `start` tracks the real first element.

In [24]:
input10 = '18,1,0,161,255,137,254,252,14,95,165,33,181,168,2,188'

def knot_hash(xs, lengths):
    start = 0
    N = len(xs)
    for skip, length in enumerate(lengths):
        xs = xs[:length][::-1] + xs[length:]
        rot = (length + skip) % N
        xs = xs[rot:] + xs[:rot]
        start += N - rot
    return xs[start % N] * xs[(start+1) % N]

xs = list(range(5))
assert knot_hash(xs, (3, 4, 1, 5)) == 12
xs = list(range(256))
knot_hash(xs, map(int, input10.split(',')))

46600

Rather than extend or adapt the function in part 1, let's just rewrite it using the more sophisticated rules detailed in part 2

In [25]:
def knot_hash(xs, lengths):
    start = 0
    N = len(xs)
    lengths = ([ord(c) for c in lengths] + [17, 31, 73, 47, 23]) * 64
    for skip, length in enumerate(lengths):
        xs = xs[:length][::-1] + xs[length:]
        rot = (length + skip) % N
        xs = xs[rot:] + xs[:rot]
        start += N - rot
    start = start % N
    xs = xs[start:] + xs[:start]
    return [functools.reduce(operator.xor, xs[p:p+16]) for p in range(0, 256, 16)]

def hexa(hash):
    return ''.join(f'{d:02x}' for d in hash)

R256 = list(range(256))
assert hexa(knot_hash(R256, '')) == 'a2582a3a0e66e6e86e3812dcb672a272'
assert hexa(knot_hash(R256, 'AoC 2017')) == '33efeb34ea91902bb2f59c9920caa6cd'
assert hexa(knot_hash(R256, '1,2,3')) == '3efbe78a8d82f29979031a4aa0b16a9d'

hexa(knot_hash(R256, input10))

'23234babdc6afa036749cfa9b597de1b'

### [Day 11](http://adventofcode.com/2017/day/11) Hex Ed
This was something new to me - but easily tackled once I'd found [this page on hexagonal grids](https://www.redblobgames.com/grids/hexagons/).

In [26]:
class hexagonal_coord:
    # Use "cube coords" which satisfy x+y+z = 0
    def __init__(self):
        self.x = self.y = self.z = 0

    def step(self, dir):
        'Step in a "hexagonal" direction.'
        dx, dy, dz = {
            'n': (0,1,-1),
            'ne':(1,0,-1),
            'se':(1,-1,0),
            's': (0,-1,1),
            'sw':(-1,0,1),
            'nw':(-1,1,0),
            }[dir]
        self.x += dx
        self.y += dy
        self.z += dz
        
    def mod(self):
        'Return the number of steps this point is from the origin.'
        return max(math.fabs(self.x), math.fabs(self.y), math.fabs(self.z))

def hexagonal_path(steps):
    p = hexagonal_coord()
    for d in steps:
        p.step(d)
        yield p

assert last(hexagonal_path('ne,ne,ne'.split(','))).mod() == 3
assert last(hexagonal_path('ne,ne,sw,sw'.split(','))).mod() == 0
assert last(hexagonal_path('ne,ne,s,s'.split(','))).mod() == 2
assert last(hexagonal_path('se,sw,se,sw,sw'.split(','))).mod() == 3

last(hexagonal_path(input(11).strip().split(','))).mod()

715.0

In [27]:
max(p.mod() for p in hexagonal_path(input(11).strip().split(',')))

1512.0

### [Day 12](http://adventofcode.com/2017/day/12) Digital Plumber
Another graph processing puzzle.

In [30]:
example12 = '''\
0 <-> 2
1 <-> 1
2 <-> 0, 3, 4
3 <-> 2, 4
4 <-> 2, 3, 6
5 <-> 6
6 <-> 4, 5
'''

def connected_components(g):
    'Generate connected components of graph `g`.'
    to_visit = set(g)
    while to_visit:
        fringe = {to_visit.pop()}
        cpt = set(fringe)
        while fringe:
            fringe = {v for f in fringe for v in g[f] if v in to_visit}
            to_visit -= fringe
            cpt |= fringe
        yield cpt

def read_graph(data):
    'Read in graph data formatted as in the example'
    return {v: neighbours for v, *neighbours in array(data)}

cpts = connected_components(read_graph(example12))
assert next(cpt for cpt in cpts if 0 in cpt) == {0, 2, 3, 4, 5, 6}

cpts = connected_components(read_graph(input(12)))
len(next(cpt for cpt in cpts if 0 in cpt))

115

Part 2 asks for the number of connected components.

In [31]:
ilen(connected_components(read_graph(input(12))))

221

### [Day 13](http://adventofcode.com/2017/day/13) Packet Scanners
Modulo arithmetic.

In [30]:
example13 = '''\
0: 3
1: 2
4: 4
6: 4
'''

def caught(layers_depths, delay=0):
    '''Return the positions where the packet is caught by the firewall

    layers_depths - the firewall scanner configurations
    delay - the delay before starting
    '''
    L = max(layers_depths)
    M = delay + L + 1
    layers_periods = {l: 2*(d-1) for l, d in layers_depths.items()} 
    return (pos for pos in range(L+1)
            if (pos + delay) % layers_periods.get(pos, M) == 0)

def trip_severity(layers_depths):
    'Calculate the severity of a trip through the firewall.'
    return sum(pos * layers_depths[pos] for pos in caught(layers_depths))

example_layers_depths = dict(array(example13))
assert trip_severity(example_layers_depths) == 24

layers_depths = dict(array(input(13)))
trip_severity(layers_depths)

1588

Part 2 asks the smallest time at which a packet will pass cleanly through the firewall.

In [31]:
def escape(layers_depths):
    'Return the smallest time when the packet passes through the firewall.'
    return next(delay for delay in its.count(1)
                if not any(1 for _ in caught(layers_depths, delay)))

assert escape(example_layers_depths) == 10
escape(layers_depths)

3865118

### [Day 14](http://adventofcode.com/2017/day/14) Disk Defragmentation
Knot hashes and graph processing.

In [32]:
def disk_grid(key_string):
    'Returns the disk layout produced by `key_string`'
    R256 = list(range(256))
    return [knot_hash(R256, f'{key_string}-{i}') for i in range(128)]

def bitcount(d):
    return bin(d).count('1')

def used_count(grid):
    return sum(bitcount(d) for d in flatten(grid))

assert used_count(disk_grid('flqrgnkx')) == 8108
used_count(disk_grid('nbysizxe'))

8216

The second part asks us to count up connected components of the grid.

In [33]:
def neighbours(v):
    return [(v[0]+x,v[1]+y) for x,y in ((0,1), (0,-1), (1,0), (-1,0))]

def connected_components(grid):
    R = range(128)
    def on(v):
        'Return True iff `v` is a used position in the grid.'
        r, c = v
        d, m = divmod(c, 8)
        return r in R and c in R and grid[r][d] & 1 << (7 - m)

    to_visit = {v for v in its.product(R, repeat=2) if on(v)}
    
    while to_visit:
        v = to_visit.pop()
        fringe = {v}
        cpt = {v}
        while fringe:
            fringe = {n for f in fringe for n in neighbours(f)
                      if on(n) and n not in (cpt|fringe)}
            cpt |= fringe
            to_visit -= fringe
        yield cpt

assert ilen(connected_components(disk_grid('flqrgnkx'))) == 1242
ilen(connected_components(disk_grid('nbysizxe')))

1139

### [Day 15](http://adventofcode.com/2017/day/15) Duelling Generators
Number crunching.

In [34]:
def generator(factor, seed, mod=1):
    while True:
        seed *= factor
        seed %= 0x7fffffff
        if seed % mod == 0:
            yield seed & 0xffff

genA = generator(16807, 65)
genB = generator(48271, 8921)

assert sum(a == b for a, b in its.islice(zip(genA, genB), 40_000_000)) == 588

genA = generator(16807, 516)
genB = generator(48271, 190)

sum(a == b for a, b in its.islice(zip(genA, genB), 40_000_000))

597

In [35]:
genA = generator(16807, 516, 4)
genB = generator(48271, 190, 8)

sum(a == b for a, b in its.islice(zip(genA, genB), 5_000_000))

303

### [Day 16](http://adventofcode.com/2017/day/16) Permutation Promenade
Shuffling a sequence according to a list of instructions.

In [36]:
def dance(progs, moves):
    progs = collections.deque(progs)
    for m in moves:
        if m[0] == 's':
            progs.rotate(int(m[1:]))
        elif m[0] == 'x':
            swap(progs, *ints(m[1:]))
        else:
            assert m[0] == 'p'
            a, _, b = m[1:].partition('/')
            swap(progs, progs.index(a), progs.index(b))
    return list(progs)

progs = 'abcde'
moves = 's1 x3/4 pe/b'.split()
permuted = dance(progs, moves)
assert ''.join(permuted) == 'baedc'

progs = 'abcdefghijklmnop'
moves = input(16).strip().split(',')
permuted = dance(progs, moves)
''.join(permuted)

'pkgnhomelfdibjac'

The second part of the puzzle requires the dance to be repeated 1e9 times. To do this, we decompose the moves into a shuffle and a swap, which can be applied independently to form the whole dance.

In [37]:
def decompose_dance(progs, moves):
    'Decompose dance moves into a positional shuffle and element swaps'
    shuf = collections.deque(range(len(progs)))
    swop = list(progs)

    for m in moves:
        if m[0] == 's':
            shuf.rotate(int(m[1:]))
        elif m[0] == 'x':
            swap(shuf, *ints(m[1:]))
        else:
            a, _, b = m[1:].partition('/')
            swap(swop, swop.index(a), swop.index(b))

    return dict(enumerate(shuf)), dict(zip(progs, swop))

[Exponentiation by squaring](https://en.wikipedia.org/wiki/Exponentiation_by_squaring) allows us to apply these rearrangements 1e9 times in milliseconds rather than hours.

In [61]:
def keep_dancing(progs, shuf, swop, repeat=1):
    'Repeat the `(suf, swap)` dance `repeat` times'
    P = len(progs)
    while repeat:
        repeat, rem = divmod(repeat, 2)
        if rem:
            progs = [swop[progs[shuf[i]]] for i in range(P)]
        shuf = {k: shuf[v] for k, v in shuf.items()}
        swop = {k: swop[v] for k, v in swop.items()}
    return progs

progs = list('abcde')
shuf, swop = decompose_dance(progs, 's1 x3/4 pe/b'.split())

assert keep_dancing(progs, shuf, swop)    == list('baedc')
assert keep_dancing(progs, shuf, swop, 2) == list('ceadb')

progs = list('abcdefghijklmnop')
shuf, swop = decompose_dance(progs, input(16).split(','))
''.join(keep_dancing(progs, shuf, swop, 1_000_000_000))

'pogbjfihclkemadn'

### [Day 17](http://adventofcode.com/2017/day/17) Spinlock
Arithmetic

In [62]:
def spinlock(step, stop):
    pos, buf = 0, [0]
    for v in range(1, stop+1):
        pos = (pos + 1 + step) % len(buf)
        buf.insert(pos, v)
    return buf

buf = spinlock(3, 2017)
i = buf.index(2017)
assert buf[(i + 1) % len(buf)] == 638

buf = spinlock(366, 2017)
i = buf.index(2017)
buf[(i + 1) % len(buf)]

1025

For the second part, growing a list by 50 million insertions would take too long. All we need, though, is the second value in the buffer -- the first value remains set to zero. Simply keep track of the length of the buffer and the position within it, updating the result when we're positioned at the start of the buffer.

In [63]:
def spinlock(step, stop):
    result = None
    pos = 0
    for length, v in enumerate(range(1, stop+1), 1):
        pos = (pos + 1 + step) % length
        if pos == 0:
            result = v
    return result

spinlock(366, 50_000_000)

37803463

### [Day 18](http://adventofcode.com/2017/day/18) Duet
Interpret assembler code.

In [41]:
example18 = '''\
set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2
'''

def execute(instructions):
    freq = None
    pc = 0
    regs = collections.defaultdict(int)

    def value(arg):
        if arg in regs:
            return regs[arg]
        else:
            return int(arg)

    while pc in instructions:
        op, *args = instructions[pc]
        if   op == 'snd': freq = value(args[0])
        elif op == 'set': regs[args[0]]  = value(args[1])
        elif op == 'add': regs[args[0]] += value(args[1])
        elif op == 'mul': regs[args[0]] *= value(args[1])
        elif op == 'mod': regs[args[0]] %= value(args[1])
        elif op == 'rcv':
            if regs.get(args[0]): return freq
        elif op != 'jgz':
            assert False, f'invalid operation: {op}'

        if op == 'jgz' and value(args[0]) > 0:
            pc += value(args[1])
        else:
            pc += 1

def parse_instructions(data):
    return  {
        i : line.split()
        for i, line in enumerate(data.splitlines())
    }

assert execute(parse_instructions(example18)) == 4

execute(parse_instructions(input(18)))

7071

For the second part, the `snd` and `rcv` instructions are implemented differently. We simulate two processes running concurrently, communicating through message queues.

In [42]:
def execute_part2(instructions, id, rcv_q, snd_q):
    pc, sent = 0, 0
    regs = collections.defaultdict(int, p=id)

    def value(arg):
        if arg in regs:
            return regs[arg]
        else:
            return int(arg)
    while pc in instructions:
        op, *args = instructions[pc]
        if   op == 'snd':
            snd_q.append(value(args[0]))
            sent += 1
        elif op == 'set': regs[args[0]]  = value(args[1])
        elif op == 'add': regs[args[0]] += value(args[1])
        elif op == 'mul': regs[args[0]] *= value(args[1])
        elif op == 'mod': regs[args[0]] %= value(args[1])
        elif op == 'rcv':
            if not rcv_q: yield sent                
            regs[args[0]] = rcv_q.popleft()

        if op == 'jgz' and value(args[0]) > 0:
            pc += value(args[1])
        else:
            pc += 1

def execute_both(instructions):
    'Run two programs and return the number of values sent.'
    qs = [collections.deque() for _ in range(2)]
    progs = [execute_part2(instructions, i, qs[i], qs[1-i]) for i in range(2)]
    sends = [0, 0]

    sends[0] = next(progs[0])
    while any(qs):
        for i in range(2):
            if qs[i]:
                sends[i] = next(progs[i])

    return sends

execute_both(parse_instructions(input(18)))[1]

8001

### [Day 19](http://adventofcode.com/2017/day/19) A Series of Tubes
Follow an ASCII path. Part 1 asks what alphabetic characters we encounter.

In [44]:
def load_route(data):
    '''Loads the route data.

    returns a dict mapping position to the character at that position
    '''
    return {(c, r): v
            for r, row in enumerate(data.splitlines())
            for c, v in enumerate(row)}

NSEW = {(0,-1), (0,1), (1,0), (-1,0)}

def step(pos, dir):
    return pos[0]+dir[0], pos[1]+dir[1]

def reverse(dir):
    return -dir[0], -dir[1]

def traverse(data, pos, dir):
    c = data.get(pos)
    while c not in '! ':
        yield c
        pos = step(pos, dir)
        c = data.get(pos, '!')
        if c == '+':
            dir = next((d for d in NSEW - {reverse(dir)}
                        if data.get(step(pos, d), '!') not in ' !'),
                       '!')

def start_pos(route):
    'Locate the start position, direction.'
    # We are told to start from the top row, moving S
    starts = {(x, y) for (x, y), c in route.items() if y == 0 and c == '|'}
    assert len(starts) == 1
    return starts.pop(), (0, 1)

example19 = '''\
     |          
     |  +--+    
     A  |  C    
 F---|----E|--+ 
     |  |  |  D 
     +B-+  +--+ 
'''

route = load_route(example19)
assert ''.join(c for c in traverse(route, *start_pos(route))
               if c in string.ascii_letters) == 'ABCDEF'

route = load_route(input(19))
''.join(c for c in traverse(route, *start_pos(route)) if c in string.ascii_letters)

'VEBTPXCHLI'

Part 2 asks for the length of the path.

In [47]:
route = load_route(example19)
assert ilen(traverse(route, *start_pos(route))) == 38
route = load_route(input(19))
ilen(traverse(route, *start_pos(route)))

18702

### [Day 20](http://adventofcode.com/2017/day/20) Particle Swarm
Newtonian motion

In [32]:
def manhattan(row):
    '''Returns "Manhattan" acceleration, velocity, position.

    This can be used as a key to sort eventual distance from origin.
    '''
    M = math.fabs
    return sum(map(M, row[6:])), sum(map(M, row[3:6])), sum(map(M, row[:3]))

example20 = '''\
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>
'''
state = array(example20)
assert min(enumerate(state), key=lambda i_row: manhattan(i_row[1]))[0] == 0

state = array(input(20))
assert len({tuple(r) for r in state}) == len(state) # Check there's a unique answer

min(enumerate(state), key=lambda i_row: manhattan(i_row[1]))[0]

344

I cheated on the second part, which asks how many particles are left after all collisions have been resolved. I simulated the universe for a (large) fixed number of steps, and guessed there would be no further collisions.

In [53]:
position = operator.itemgetter(0, 1, 2)

def step(state):
    for row in state:
        row[3]+=row[6]; row[4]+=row[7]; row[5]+=row[8] 
        row[0]+=row[3]; row[1]+=row[4]; row[2]+=row[5] 

example20_part2 = '''\
p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>
'''

def simulate(state):
    while True:
        state.sort(key=position)
        state = [list(gp) for _, gp in its.groupby(state, key=position)]
        state = [gp[0] for gp in state if len(gp) == 1]
        step(state)
        yield len(state)

alive = simulate(initial_state(example20_part2))
assert next(alive) == 4
assert next(alive) == 4
assert next(alive) == 1

alive = simulate(initial_state(input(20)))
next(its.islice(alive, 10000, None)) # HACK: Guess no collisions for t > 10000

404

### [Day 21](http://adventofcode.com/2017/day/21) Fractal Art
Growing a pattern.

In [56]:
def side(sq):
    'Return the side length of a square.'
    return int(math.sqrt(len(sq)))

def rotate(sq):
    'Rotate a square by a right angle.'
    N = side(sq)
    return {(N-y-1,x):v for (x,y),v in sq.items()}

def rotations(sq):
    'Generate the 4 rotational variants of a square.'
    for _ in range(4):
        yield sq
        sq = rotate(sq)

def flip(sq):
    'Flip a square about x==y diagonal.'
    return {(y,x):v for (x,y),v in sq.items()}

def variants(sq):
    'Yield the rotational and flipped variants of a square.'
    return its.chain(rotations(sq), rotations(flip(sq)))

def hashable_sq(sq):
    '''Return a hashable version of the square dict.

    This is for use as a key when looking up patterns.
    '''
    return tuple(sorted(sq.items()))

def sq_points(S):
    'Generate (x, y) positions in a square.'
    return its.product(range(S), repeat=2)

def load(pattern):
    '''Loads a pattern of the form ".#./..#/###"

    Returns a dict mapping (x, y) => True if the entry at (x, y) is #
                                     False otherwise
    '''
    return {(x, y): (v == '#')
            for y, row in enumerate(pattern.split('/'))
            for x, v in enumerate(row)}

def load_all(data):
    '''Load all of the patterns

    Returns a dict mapping pattern to replacement
    '''
    replacements = {}

    for line in data.splitlines():
        lt, _, rt = line.partition(' => ')
        patt = load(lt)
        repl = load(rt)
        replacements.update(
            (p, repl) for p in map(hashable_sq, variants(patt)))

    return replacements

def shift(sq, X, Y, S):
    return {(x, y): sq[(X+x,Y+y)] for x, y in sq_points(S)}

def fill(sq, X, Y, repl):
    'Fills square `sq` with values in `repl` shifted by `(X, Y)`'
    sq.update(((X+x, Y+y), v) for (x, y), v in repl.items())

def grow(seed, rules):
    '''Yield successive generations of the seed pattern

    - If the size is evenly divisible by 2, break the pixels up into
      2x2 squares, and replace each 2x2 square by a 3x3 square.

    - Otherwise, the size is evenly divisible by 3; break the pixels
      up into 3x3 squares, and replace each 3x3 square by a 4x4 square
    '''
    rule_sizes = (2, 3)
    while True:
        yield seed
        N = side(seed)
        new_seed = {}
        R = next(R for R in rule_sizes if N % R == 0)
        for x, y in sq_points(N//R):
            match = shift(seed, R*x, R*y, R)
            fill(new_seed, (R+1)*x, (R+1)*y, rules[hashable_sq(match)])
        seed = new_seed

def pixels(sq):
    'Return the number of pixels which are on in `sq`.'
    return sum(sq.values())

def display(sq):
    N = side(sq)
    return '\n'.join(
        ''.join('.#'[sq[(y,x)]] for y in range(N))
        for x in range(N))

example21 = '''\
../.# => ##./#../...
.#./..#/### => #..#/..../..../#..#
'''

seed = load('.#./..#/###')
rules = load_all(example21)

g = grow(seed, rules)
print('\n' + display(next(g)))
print('\n' + display(next(g)))
print('\n' + display(next(g)))

assert pixels(next(its.islice(grow(seed, rules), 2, None))) == 12


.#.
..#
###

#..#
....
....
#..#

##.##.
#..#..
......
##.##.
#..#..
......


Part one: how many pixels stay on after 5 iterations?

In [58]:
rules = load_all(input(21))
pixels(next(its.islice(grow(seed, rules), 5, None)))

142

Part two: how many pixels stay on after 18 iterations?

In [60]:
pixels(next(its.islice(grow(seed, rules), 18, None)))

1879071

### [Day 22](http://adventofcode.com/2017/day/22) Sporifica Virus
Updating a grid according to simple rules.

In [61]:
CLEAN, WEAKENED, INFECTED, FLAGGED = range(4)

def carry_virus(grid, pos, dir):
    while True:
        status = grid.get(pos, CLEAN)
        yield pos, dir, status
        infected = status == INFECTED
        dir *= -1j if infected else 1j
        grid[pos] = CLEAN if infected else INFECTED
        pos += dir

def load(data):
    '''Loads the data into a grid

    Returns:
    - the grid mapping (complex) position to infected status
    - the initial position
    - the initial direction
    '''
    rows = data.splitlines()
    mid = (len(rows[0])-1)//2, -(len(rows)-1)//2
    return {complex(x, -y): (INFECTED if c == '#' else CLEAN)
            for y, row in enumerate(rows)
            for x, c in enumerate(row)}, complex(*mid), 1j

example22 = '''\
..#
#..
...
'''

grid, pos, dir = load(example22)
g = carry_virus(grid, pos, dir)
print(next(g))
print(next(g))
print(next(g))

def infections_caused(data, bursts):
    grid, pos, dir = load(data)
    steps = its.islice(carry_virus(grid, pos, dir), bursts)
    return sum(1 for pos, dir, inf in steps if inf == CLEAN)

assert infections_caused(example22, 7) == 5
assert infections_caused(example22, 70) == 41
assert infections_caused(example22, 10000) == 5587

infections_caused(input(22), 10000)

((1-1j), 1j, 0)
(-1j, (-1+0j), 2)
(0j, 1j, 0)


5406

Part two changes the rules.

In [62]:
def carry_resistant_virus(grid, pos, dir):
    while True:
        status = grid.get(pos, CLEAN)
        yield pos, dir, status
        dir *= {
            CLEAN: 1j,
            WEAKENED: 1,
            INFECTED: -1j,
            FLAGGED: -1
        }[status]
        grid[pos] = (status + 1) % 4
        pos += dir

def resistant_infections_caused(data, bursts):
    grid, pos, dir = load(data)
    steps = its.islice(carry_resistant_virus(grid, pos, dir), bursts)
    return sum(1 for pos, dir, inf in steps if inf == WEAKENED)

assert resistant_infections_caused(example22, 100) == 26
assert resistant_infections_caused(example22, 10_000_000) == 2511944
resistant_infections_caused(input(22), 10_000_000)

2511640

### [Day 23](http://adventofcode.com/2017/day/23) Coprocessor Conflagration
Implement then optimise some assembler.

In [64]:
def execute(instructions):
    pc = 0
    regs = {c : 0 for c in 'abcdefgh'}

    def value(arg):
        if arg in regs:
            return regs[arg]
        else:
            return int(arg)

    while pc in instructions:
        op, *args = instructions[pc]
        if   op == 'set': regs[args[0]]  = value(args[1])
        elif op == 'sub': regs[args[0]] -= value(args[1])
        elif op == 'mul': regs[args[0]] *= value(args[1])
        elif op != 'jnz':
            assert False, f'invalid operation: {op}'

        if op == 'jnz' and value(args[0]) != 0:
            pc += value(args[1])
        else:
            pc += 1
        yield op, regs

def parse_instructions(data):
    return  {
        i : line.split()
        for i, line in enumerate(data.splitlines())
    }

instructions = parse_instructions(input(23))

sum(1 for op, _ in execute(instructions) if op == 'mul')

3025

Part two sets register `a` to 1 and warns us the calculation is going to require optimisation. I tried putting print statements in the `execute()` function and looking for patterns in the register values, but nothing leapt out at me.

Next I tried carefully converting the assembler code to Python.

In [3]:
def day23():
    a = 1
    b = c = d = f = g = h = 0

    b = 57                   # set b 57
    c = b                    # set c b
    if a != 0:               # jnz a 2
                             # jnz 1 5
        b *= 100             # mul b 100
        b += 100000          # sub b -100000
        c = b                # set c b
        c += 17000           # sub c -17000
    while True:
        f = 1                # set f 1
        d = 2                # set d 2
        while True:
            e = 2            # set e 2
            while True:
                g = d        # set g d
                g *= e       # mul g e
                g -= b       # sub g b
                if g == 0:   # jnz g 2
                    f = 0    # set f 0
                e += 1       # sub e -1
                g = e        # set g e
                g -= b       # sub g b
                if g == 0: break # jnz g -8
            d += 1           # sub d -1
            g = d            # set g d
            g -= b           # sub g b
            if g == 0: break # jnz g -13
        if f == 0:           # jnz f 2
            h += 1           # sub h -1
        g = b                # set g b
        g -= c               # sub g c
        if g == 0:           # jnz g 2
            break            # jnz 1 3
        b += 17              # sub b -17
                             # jnz 1 -23
    return h

We are only interested in the final value of `h`, which counts the number of times `f` is set to `0` after executing a nested inner loop.

I refactored the code to use `range()` and some other simplifications.

In [6]:
def day23():
    h = 0
    b = 105700
    c = 122700

    for b in range(b, c + 17, 17):
        f = 1
        for d in range(2, b):
            for e in range(2, b):
                if d * e == b:
                    f = 0
        if f == 0:
            h += 1

    return h

So, `b` steps through a range of values. For each value of `b` we check all pairs of values between `2` and `b` to see if their product is equal to `b`. If this is `True` for any such pair, we increment the result, `h`. In other words, we're counting the composite numbers in `range(105700, 122700 + 17, 17)`.

In [12]:
def day23():
    h = 0
    b = 105700
    c = 122700

    for b in range(b, c + 17, 17):
        for d in range(2, int(math.sqrt(b))):
            if b % d == 0:
                h += 1
                break

    return h

day23()

915

### [Day 24](http://adventofcode.com/2017/day/24) Electromagnetic Moat
What's the stongest bridge you can make by connecting components?

In [33]:
def bridges(pins):
    '''Generate strengths for bridges built from `pins`
    '''
    # Each state in the space we want to explore has:
    # - current end value
    # - strength 
    # - pins left to use
    explore = [(0, 0, pins)]
    while explore:
        end, strength, pins = explore.pop()
        yield strength
        explore.extend((p[1] if p[0]==end else p[0],
                        strength + sum(p), 
                        pins[:i] + pins[i+1:])
                        for i, p in enumerate(pins) if end in p)

example24 = '''\
0/2
2/2
2/3
3/4
3/5
0/1
10/1
9/10
'''

assert max(bridges(array(example24))) >= 31
max(bridges(array(input(24))))

1868

The second part of the puzzle asks for the strength of the longest bridge you can make. If you can make multiple bridges of the longest length, pick the strongest one. To do this, adapt the state of nodes in the search space to include length information.

In [21]:
def bridges(pins):
    '''Generate (length, strength) pairs for bridges built from `pins`
    '''
    # Each state in the space we want to explore has:
    # - current end value
    # - length
    # - strength 
    # - pins left to use
    explore = [(0, 0, 0, pins)]
    while explore:
        end, length, strength, pins = explore.pop()
        yield length, strength
        explore.extend((p[1] if p[0]==end else p[0],
                        length + 1,
                        strength + sum(p), 
                        pins[:i] + pins[i+1:])
                        for i, p in enumerate(pins) if end in p)

max(bridges(load(input(24))))[1]

1841

### [Day 25](http://adventofcode.com/2017/day/25) The Halting Problem
Implement a Turing machine.

In [22]:
example25 = '''\
Begin in state A.
Perform a diagnostic checksum after 6 steps.

In state A:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state B.
  If the current value is 1:
    - Write the value 0.
    - Move one slot to the left.
    - Continue with state B.

In state B:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the left.
    - Continue with state A.
  If the current value is 1:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state A.
'''

Much of the difficulty here is parsing the input. We can use regular expressions to read the initial setup, and also the state machine.

In [23]:
state_pattern = '''\
In state (\w):
  If the current value is 0:
    - Write the value (\d)\.
    - Move one slot to the (right|left)\.
    - Continue with state (\w)\.
  If the current value is 1:
    - Write the value (\d)\.
    - Move one slot to the (right|left)\.
    - Continue with state (\w)\.
'''

State = collections.namedtuple('State', 'write step next')

def state_machine(data):
    'Create the state machine from the text description.'
    def make_state(m):
        return (
            State(int(m[0]), 1 if m[1] == 'right' else -1, m[2]),
            State(int(m[3]), 1 if m[4] == 'right' else -1, m[5]))
    return {s: make_state(m)
            for s, *m in re.compile(state_pattern).findall(data)}

setup_pattern = '''\
Begin in state (\w)\.
Perform a diagnostic checksum after (\d+) steps.
'''

def setup(data):
    'Return (start state, steps until checksum).'
    m = re.compile(setup_pattern).match(data)
    return m.group(1), int(m.group(2))

Running the machine is straightforward.

In [24]:
def run(state_machine, state, steps):
    '''Run `state_machine` for a number of `steps` starting in a given `state`.
    
    Returns the final tape
    '''
    tape = collections.defaultdict(int)
    pos = 0
    for _ in range(steps):
        action = state_machine[state][tape[pos]]
        tape[pos] = action.write
        pos += action.step
        state = action.next
    return tape

tape = run(state_machine(example25), *setup(example25))
assert sum(tape.values()) == 3

spec = input(25)
tape = run(state_machine(spec), *setup(spec))
sum(tape.values())

4217

![That was fun](done.png "50 Stars")